In [ ]:
import os
import PyPDF2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ---------- Step 1: Read Job Description ----------
with open("job_description.txt", "r", encoding="utf-8") as f:
    job_description = f.read()

# ---------- Step 2: Read Resumes from 'resumes' folder ----------
resume_texts = []
resume_files = []

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file"""
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            if page.extract_text():
                text += page.extract_text()
    return text

# Loop through all resumes in the 'resumes' folder
for file in os.listdir("resumes"):
    if file.endswith(".pdf"):
        path = os.path.join("resumes", file)
        text = extract_text_from_pdf(path)
        resume_texts.append(text)
        resume_files.append(file)

# ---------- Step 3: Vectorize (TF-IDF) ----------
documents = [job_description] + resume_texts
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(documents)

# ---------- Step 4: Compute Cosine Similarity ----------
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# ---------- Step 5: Save Results ----------
results = pd.DataFrame({
    "Resume": resume_files,
    "Score": cosine_similarities
})

# Sort by highest score
results = results.sort_values(by="Score", ascending=False)

# Save to Excel and CSV
results.to_excel("ranked_resumes.xlsx", index=False)
results.to_csv("ranked_resumes.csv", index=False)

print("✅ Screening complete! Results saved in ranked_resumes.xlsx and ranked_resumes.csv")